In [1]:
import re
import os
import itertools
from progressbar import ProgressBar
import numpy as np
from bs4 import BeautifulSoup
from sklearn.metrics import precision_recall_fscore_support,f1_score
from collections import Counter as c
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix

In [2]:
class topnclass:
    def __init__(self):
        self.labels=[]                      #parsing the required data about labels and documents
        self.data=[]
        soup=BeautifulSoup(open('input/wikipediaMediumOriginalLSHTCv3-train.txt','r',encoding='utf8'),'html.parser')
        for x in list(soup.find_all('labels')):
            self.labels.append([ int(lab) for lab in x.get_text().split()])
            self.data.append(x.next_sibling.strip('\n'))
    def toplabels(self,n):
        alllabels=list(itertools.chain.from_iterable(self.labels))###combining list of lists to find top n labels among all label ids
        label_counts=c(alllabels)
        label_top=[x[0]for x in label_counts.most_common(n)]## returning top n labels
        return label_top
    def createdatatop(self,n):
        pbar=ProgressBar()
        label_list=self.toplabels(n)
        dict_for_labels_bools={x:0for x in range(445730)}
        for el in label_list:           #########using dict to create the data fastly
            dict_for_labels_bools[el]=1
        self.label_list_all=label_list
        ############# discard only labels not data
        tmp=[]
        y=[]###########target class
        X=[]#############documents
        with open('output/top'+str(n)+'classdata.txt','w',encoding='utf-8') as f:
            for i in pbar(range(len(self.data))):
                count=0              ##########to check if a doc has labels among top n labels
                tmp1=[]
                for j in self.labels[i]:
                    if dict_for_labels_bools[j]==1:
                        count = count+1
                        f.write('__label__')
                        tmp1.append(j)
                        f.write(str(j)+' ')
                if tmp1:
                    y.append(tmp1)
                if count>0:
                    f.write(self.data[i])
                    X.append(self.data[i])
                    f.write('\n')
        return X,y
    def fastontop(self,n,loss='hs'):
        pbar=ProgressBar()
        X,y=self.createdatatop(n)
        X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.30, random_state=30,shuffle=True)
        with open('output/top'+str(n)+'classtrain.txt','w',encoding='utf-8') as f:
            for i in range(len(X_train)):
                for j in range(len(y_train[i])):
                    f.write("__label__")
                    f.write(str(y_train[i][j])+" ")
                f.write(X_train[i])
                f.write("\n")
        with open('output/top'+str(n)+'classtest.txt','w',encoding='utf-8') as f:
            for i in range(len(X_test)):
                f.write(X_test[i])
                f.write("\n")
        cmd1="fasttext supervised  -dim 200 -epoch 100 -lr 0.25 -loss "+loss+" -input top"+str(n)+"classtrain.txt -output top"+str(n)+"classmodel"
        mod="output/top"+str(n)+"classmodel.bin "
        tes="output/top"+str(n)+"classtest.txt "
        res="output/top"+str(n)+"classresult.txt"
        y_train_labellength=[len(x) for x in y_train]
        Average_labels_per_document = sum(y_train_labellength)/len(y_train_labellength)
        print('Average_labels_per_document in top '+str(n)+' class data is',Average_labels_per_document)
        label_predict_count=str(round(Average_labels_per_document))
        print('So,Predicting ',label_predict_count,'labels using fasttext')
        cmd2="fasttext predict "+mod+ tes+label_predict_count+' > '+ res
        os.system(cmd1)
        os.system(cmd2)
        f = open(res, 'r')###############reading predicted values
        lines =[re.split('__label__', line) for line in f.readlines()]
        f.close()
        lines=[x[1:]for x in lines]
        lines=[list(map(int, x)) if isinstance(x,list) else [int(x)] for x in lines]
        y_true_row=[]###############################################generating sparse matrices of predicted and true labels for test docs
        y_true_col=[]
        y_true_data=[]
        y_pred_row=[]
        y_pred_col=[]
        y_pred_data=[]
        dict_for_label_indexes={}#############################using dict to get indexes of labels fastly
        counter_for_index_for_labels=0
        for x in self.label_list_all:
            dict_for_label_indexes[x]=counter_for_index_for_labels
            counter_for_index_for_labels=counter_for_index_for_labels+1
        for x in pbar(range(len(y_test))):
            t1=lines[x]
            t2=y_test[x]
            for k in range(len(t1)):
                y_pred_row.append(x)
                index1=dict_for_label_indexes[t1[k]]
                y_pred_col.append(index1)
                y_pred_data.append(1)
            for z in range(len(t2)):
                y_true_row.append(x)
                index2=dict_for_label_indexes[t2[z]]
                y_true_col.append(index2)
                y_true_data.append(1)
        y_pred_sparse=csr_matrix((y_pred_data, (y_pred_row, y_pred_col)),shape=(len(y_test),n))
        y_true_sparse=csr_matrix((y_true_data, (y_true_row, y_true_col)),shape=(len(y_test),n))
        print("For top "+str(n)+" class (precision,recall,fscore) using macro average:",precision_recall_fscore_support(y_true_sparse,y_pred_sparse, average='macro'))


In [3]:
obj=topnclass()

In [4]:
obj.fastontop(10,'hs')   ##### pass 'softmax' in place of 'hs' to use softmax or leave it to use hierarchical softmax

100% |########################################################################|


Average_labels_per_document in top 10 class data is 1.0369938946015425
So,Predicting  1 labels using fasttext


IndexError: list index out of range

In [5]:
obj.fastontop(10,'softmax')

100% |########################################################################|


Average_labels_per_document in top 10 class data is 1.0369938946015425
So,Predicting  1 labels using fasttext


IndexError: list index out of range